In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
!pip install transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model

     |████████████████████████████████| 2.5MB 13.2MB/s 
     |████████████████████████████████| 901kB 54.9MB/s 
     |████████████████████████████████| 3.3MB 70.0MB/s 


In [ ]:
csv = "/content/drive/MyDrive/commonlitreadabilityprize/train.csv"
data = pd.read_csv(csv)
df = pd.DataFrame(data)
df = df.drop(['url_legal', 'license', 'id', 'standard_error'], axis=1)
df

,excerpt,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118
3,And outside before the palace a great garden w...,-1.054013
4,Once upon a time there were Three Bears who li...,0.247197
...,...,...
2829,When you think of dinosaurs and where they liv...,1.711390
2830,So what is a solid? Solids are usually hard be...,0.189476
2831,The second state of matter we will discuss is ...,0.255209
2832,Solids are shapes that you can actually touch....,-0.215279


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
max_seq_length = 313
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
def preprocess_input(tokenizer, data):
  tokenized_data = [tokenizer.tokenize(text) for text in data]
  for tokens in tokenized_data:
    tokens = ["[CLS]"] + tokens

  input_ids = [get_ids(tokens, tokenizer, max_seq_length) for tokens in tokenized_data]
  input_masks = [get_masks(tokens, max_seq_length) for tokens in tokenized_data]
  input_segments = [get_segments(tokens, max_seq_length) for tokens in tokenized_data]
  
  res = (np.asarray(input_ids),
      np.asarray(input_masks),
      np.asarray(input_segments))
  return res

def convert_output_to_numpy_array(output):
  return np.asarray(output)


In [ ]:
def build_model():
  input_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name='input_ids')
  input_masks = Input(shape=(max_seq_length,), dtype=tf.int32, name='input_masks')
  input_segments = Input(shape=(max_seq_length,), dtype=tf.int32, name='input_segments')

  bert_model = TFBertModel.from_pretrained('bert-base-uncased')

  sequence_output = bert_model(input_ids=input_ids, attention_mask=input_masks, token_type_ids=input_segments)[0]
  cls_output = sequence_output[:, 0, :]
  dropout = Dropout(.3)(cls_output)
  final_layer = Dense(1, activation='linear')(dropout)

  model = Model(inputs=[input_ids, input_masks, input_segments], outputs=final_layer)
  return model

In [ ]:
inputs = preprocess_input(tokenizer, df['excerpt'])
outputs = convert_output_to_numpy_array(df['target'])

In [ ]:
x_train = [train_test_split(data, test_size=0.15, random_state=42)[0] for data in inputs]
x_val = [train_test_split(data, test_size=0.15, random_state=42)[1] for data in inputs]
y_train, y_val = train_test_split(outputs, test_size=0.15, random_state=42)
model=build_model()
model.compile(tf.keras.optimizers.Adam(lr=1e-4), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_val, y_val))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


151/151 [==============================] - 76s 394ms/step - loss: 0.7981 - root_mean_squared_error: 0.8934 - val_loss: 0.5685 - val_root_mean_squared_error: 0.7540
Epoch 2/10
151/151 [==============================] - 57s 375ms/step - loss: 0.3194 - root_mean_squared_error: 0.5651 - val_loss: 0.3159 - val_root_mean_squared_error: 0.5621
Epoch 3/10
151/151 [==============================] - 57s 375ms/step - loss: 0.2074 - root_mean_squared_error: 0.4554 - val_loss: 0.2996 - val_root_mean_squared_error: 0.5474
Epoch 4/10
151/151 [==============================] - 57s 375ms/step - loss: 0.1573 - root_mean_squared_error: 0.3966 - val_loss: 0.2758 - val_root_mean_squared_error: 0.5252
Epoch 5/10
151/151 [==============================] - 57s 375ms/step - loss: 0.1062 - root_mean_squared_error: 0.3259 - val_loss: 0.3516 - val_root_mean_squared_error: 0.5930
Epoch 6/10
151/151 [==============================] - 57s 375ms/step - loss: 0.0743 - root_mean_squared_error: 0.2726 - val_loss: 0.3490

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(6, True, 1)
for train_index, val_index in kfold.split(df['excerpt']):
  x_train = [data[train_index] for data in inputs]
  x_val = [data[val_index] for data in inputs]
  y_train = outputs[train_index]
  y_val = outputs[val_index]
  K.clear_session()
  model=build_model()
  model.compile(tf.keras.optimizers.Adam(lr=1e-4), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
  model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_val, y_val))

In [ ]:
csv = "/content/drive/MyDrive/commonlitreadabilityprize/test.csv"
data = pd.read_csv(csv)
df2 = pd.DataFrame(data)
df2 = df2.drop(['url_legal', 'license', 'excerpt'], axis=1)

In [ ]:
test_inputs = preprocess_input(tokenizer, df2['excerpt'])
df2['target'] = model.predict(test_inputs)

array([[-0.51325905, -0.46205387, -0.333265  , -1.8931178 , -1.7205174 ,
        -0.58148104,  0.46195298]])